## Analise Complementar

In [ ]:
# Matriz de correlacao
df2corr = df1[['anual_rate','age','thickness10','thickness','curv', 'sulc',
              'thickness_layer_1','thickness_layer_2','thickness_layer_3',
              'thickness_layer_4','thickness_layer_5','thickness_layer_6',
              'bigbrain_layer_1','bigbrain_layer_2','bigbrain_layer_3',
              'bigbrain_layer_4','bigbrain_layer_5','bigbrain_layer_6']]

f = plt.figure(figsize=(19, 15))
s = 14
data = df2corr.corr()
plt.matshow(data, fignum=f.number)
for (x, y), value in np.ndenumerate(data):
    plt.text(x, y, f"{value:.2f}", va="center", ha="center")
plt.xticks(range(df2corr.select_dtypes(['number']).shape[1]), df2corr.select_dtypes(['number']).columns, fontsize=s, rotation=45)
plt.yticks(range(df2corr.select_dtypes(['number']).shape[1]), df2corr.select_dtypes(['number']).columns, fontsize=s)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=s)
plt.title('Correlation Matrix', fontsize=16);
olt.show()

# Boruta
X = df1.drop(columns =['anual_rate', 'structure_name','lobe_name']) 
y = df1.anual_rate

forest = RandomForestRegressor(n_jobs=-1, max_depth=20)
forest.fit(X, y)

feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2, random_state=1, alpha= 0.001)
feat_selector.fit(X.to_numpy(), y)

feature_ranks = list(zip(X.columns, 
                         feat_selector.ranking_, 
                         feat_selector.support_,
                         forest.feature_importances_
                        ))
for feat in feature_ranks:
    print('Feature: {:<25} Rank: {},  Keep: {}, Importance: {}'.format(feat[0], feat[1], feat[2], feat[3]))

In [ ]:
list_avg_rate = []

for i in df1.structure_name.unique():
    
    df5 = df1[df1.structure_name==i]
    
    avg_rate1 = df5[df1.age<=30].anual_rate.mean()
    avg_rate2 = df5[(df1.age<60) & (df1.age>30)].anual_rate.mean()
    avg_rate3 = df5[df1.age>=60].anual_rate.mean()
    
    t10 = df5.thickness10.mean()
    curv = df5.curv.mean()
    sulc = df5.sulc.mean()
    
    l1 = df5.thickness_layer_1.mean()
    b1 = df5.bigbrain_layer_1.mean()
    l2 = df5.thickness_layer_2.mean()
    b2 = df5.bigbrain_layer_2.mean()
    l3 = df5.thickness_layer_3.mean()
    b3 = df5.bigbrain_layer_3.mean()
    l4 = df5.thickness_layer_4.mean()
    b4 = df5.bigbrain_layer_4.mean()
    l5 = df5.thickness_layer_5.mean()
    b5 = df5.bigbrain_layer_5.mean()
    l6 = df5.thickness_layer_6.mean()
    b6 = df5.bigbrain_layer_6.mean()
    
    list_avg_rate.append([i,
                          avg_rate1,avg_rate2,avg_rate3,
                          t10, curv,sulc,
                          b1,l1,
                          b2,l2,
                          b3,l3,
                          b4,l4,
                          b5,l5,
                          b6,l6
                         ])

df5 = pd.DataFrame(list_avg_rate)#.sort_values(by=2)
df5.columns = ['structure_name',
               'avg_rate1', 'avg_rate2', 'avg_rate3',
               'thickness10', # resultados do trabalho do bruno
               'curv', 'sulc',
               'bigbrain_layer_1', 
               'thickness_layer_1', # resultados sofrem interferencia da espessura inicial
               'bigbrain_layer_2', 
               'thickness_layer_2',
               'bigbrain_layer_3', 
               'thickness_layer_3',
               'bigbrain_layer_4', 
               'thickness_layer_4',
               'bigbrain_layer_5', 
               'thickness_layer_5',
               'bigbrain_layer_6', 
               'thickness_layer_6'
               ]

df5['dif2'] = df5.avg_rate1*df5.avg_rate1 - df5.avg_rate3*df5.avg_rate3*10
        
for test in df5.columns[4:-1]:

    corr = np.corrcoef(df5[test],df5.avg_rate1)[0, 1]
    sns.regplot(x=test,y='avg_rate1', data=df5).set(
        title="correlation first 30 years ="+ str(np.round(corr,2)))
    plt.show()

    corr = np.corrcoef(df5[test],df5.avg_rate2)[0, 1]
    sns.regplot(x=test,y='avg_rate2', data=df5).set(
        title="correlation second 30 years ="+ str(np.round(corr,2)))
    plt.show()

    corr = np.corrcoef(df5[test],df5.avg_rate3)[0, 1]
    sns.regplot(x=test,y='avg_rate3', data=df5).set(
        title="correlation third 30 years ="+ str(np.round(corr,2)))
    plt.show()
    
    corr = np.corrcoef(df5[test],df5.dif2)[0, 1]
    sns.regplot(x=test,y='dif2', data=df5).set(
    title="correlação diferença do afinamento em desenvolvimento e velhice ="+ str(np.round(corr,2)))
    plt.show()

df5.corr()

In [ ]:
# calculando teste estatistico
df5_n = df5.copy()
df5_n['avg_rate1'] = df5['avg_rate1']/(df5['avg_rate1'].mean())
df5_n['avg_rate2'] = df5['avg_rate2']/(df5['avg_rate2'].mean())
df5_n['avg_rate3'] = df5['avg_rate3']/(df5['avg_rate3'].mean())

df5_n['dif2'] = df5_n.avg_rate1*df5_n.avg_rate1 - df5_n.avg_rate3*df5_n.avg_rate3
#df5_n['dif2'] = df5_n.avg_rate1 - df5_n.avg_rate3

x1 = -0.4 #[0.4;0.7]
x2 = -x1
g1 = df5_n[df5_n.dif2<x1]
g2 = df5_n[(df5_n.dif2>x1) & (df5_n.dif2<x2)]
g3 = df5_n[df5_n.dif2>x2]

print(len(g1), len(g2), len(g3))
print(g1.bigbrain_layer_1.mean(), g2.bigbrain_layer_1.mean(), g3.bigbrain_layer_1.mean())
from scipy import stats 
print(stats.kruskal(g1.bigbrain_layer_1,g2.bigbrain_layer_1,g2.bigbrain_layer_1)[1])
print(stats.f_oneway(g1.bigbrain_layer_1,g2.bigbrain_layer_1,g2.bigbrain_layer_1)[1])

# criando imagens
df5_n['dif2'] = df5_n.avg_rate1*df5_n.avg_rate1 - df5_n.avg_rate3*df5_n.avg_rate3
df5_n = df5_n.sort_values(by = 'dif2')
plt.figure(figsize=(16, 4), dpi=80)
plt.plot(df5_n.structure_name.astype(str), df5_n.avg_rate1*df5_n.avg_rate1, '--o') 
plt.plot(df5_n.structure_name.astype(str), df5_n.avg_rate3*df5_n.avg_rate3, '--o')
plt.xticks(rotation = -45) 
plt.show()

plt.figure(figsize=(16, 4), dpi=80)
corr = np.corrcoef(df5_n['bigbrain_layer_1'],df5_n.dif2)[0, 1]
sns.regplot(x='bigbrain_layer_1',y='dif2', data=df5_n).set(
    title="correlação entre a diferença do afinamento em desenvolvimento e velhice e o percentual da camada 1 ="+ str(np.round(corr,3)))
plt.show()

df5_n['dif2'] = df5_n.avg_rate1 - df5_n.avg_rate3
df5_n = df5_n.sort_values(by = 'dif2')
plt.figure(figsize=(16, 4), dpi=80)
plt.plot(df5_n.structure_name.astype(str), df5_n.avg_rate1, '--o') 
plt.plot(df5_n.structure_name.astype(str), df5_n.avg_rate3, '--o')
plt.xticks(rotation = -45) 
plt.show()

plt.figure(figsize=(16, 4), dpi=80)
corr = np.corrcoef(df5_n['bigbrain_layer_1'],df5_n.dif2)[0, 1]
sns.regplot(x='bigbrain_layer_1',y='dif2', data=df5_n).set(
    title="correlação entre a diferença do afinamento em desenvolvimento e velhice e o percentual da camada 1 ="+ str(np.round(corr,3)))
plt.show()